In [1]:
import math
from tensorboardX import SummaryWriter
sess = None

In [2]:
import tensorflow as tf
import collections
gpu_options = tf.GPUOptions(allow_growth=True,per_process_gpu_memory_fraction=0.8)
tf.reset_default_graph()
sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

In [3]:
from wrappers import make_atari_deepmind
import gym
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
env_name = "LunarLanderContinuous-v2"
#env_name = "Pendulum-v0"
#env_name = "BreakoutNoFrameskip-v4"
#env_name = "SpaceInvadersNoFrameskip-v4"
env = gym.make(env_name)
env_eval = gym.make(env_name)
observation_space = env.observation_space
action_space = env.action_space
print(observation_space)
print(action_space)
print(action_space.low)
print(action_space.high)

Box(8,)
Box(2,)
[-1. -1.]
[1. 1.]


/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [4]:

from ddpgagent import DDPGAgent
import tr_helpers
import networks
import models

#agent.epsilon = 0.5

dddqn_config = {
    'GAMMA' : 0.99,
    'ACTOR_LEARNING_RATE' : 1e-4,
    'CRITIC_LEARNING_RATE' : 1e-3,
    'STEPS_PER_EPOCH' : 1,
    'BATCH_SIZE' : 32,
    'EPSILON' : 1,
    'MIN_EPSILON' : 0.001,
    'EPSILON_DECAY_FRAMES' : 200000,
    'NAME' : 'DDPG',
    'SCORE_TO_WIN' : 200,
    'NUM_STEPS_FILL_BUFFER' : 10000,
    'REPLAY_BUFFER_TYPE' : 'normal',
    'REPLAY_BUFFER_SIZE' : 100000,
    'STEPS_NUM' : 1,
    'NETWORK' : models.ModelDDPG(networks.default_ddpg_network),
    'REWARD_SHAPER' : tr_helpers.DefaultRewardsShaper(scale_value = 1.0/10.0),
    'EPISODES_TO_LOG' : 10,
    'FRAMES_TO_LOG' : 5*1024, 
    'LIVES_REWARD' : 1,
    'ATOMS_NUM' : 1
    }


agent = DDPGAgent(env, env_eval, sess, env_name, observation_space, action_space, config = dddqn_config)
#agent.restore('nn/pong_dddqn_config0PongNoFrameskip-v4')
#agent.epsilon = 0.02

W1013 20:50:12.633558 139620787271488 deprecation_wrapper.py:119] From /home/trrrrr/Documents/github/ml/dqn_atari/ddpgagent.py:106: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1013 20:50:12.637350 139620787271488 deprecation_wrapper.py:119] From /home/trrrrr/Documents/github/ml/dqn_atari/networks.py:586: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W1013 20:50:12.637768 139620787271488 deprecation.py:323] From /home/trrrrr/Documents/github/ml/dqn_atari/networks.py:588: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.
W1013 20:50:12.640220 139620787271488 deprecation.py:506] From /home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in 

In [ ]:
env.reset()
agent.train()

Frames per seconds:  267.80798189841204
Frames per seconds:  287.93813580258086
Frames per seconds:  283.71368625426777
Frames per seconds:  285.0558322767825
Frames per seconds:  273.3287930497718
mean_rewards:  -383.7793413313276 mean_steps:  195.8
saving next best rewards:  -383.7793413313276
Frames per seconds:  279.9167852873314
Frames per seconds:  265.5680677781862
Frames per seconds:  259.4807744729389
Frames per seconds:  259.05680625220117
Frames per seconds:  258.7601862219966
mean_rewards:  -398.63851842236454 mean_steps:  302.4
Frames per seconds:  264.33356084451
Frames per seconds:  257.0266071541313
Frames per seconds:  264.8411804718007
Frames per seconds:  270.0848982341281
Frames per seconds:  246.982864031649
mean_rewards:  -216.17119487230266 mean_steps:  260.7
saving next best rewards:  -216.17119487230266
Frames per seconds:  263.3327583908756
Frames per seconds:  247.29754387621554
Frames per seconds:  242.09706559775628
Frames per seconds:  267.2674305118058
Fr

Frames per seconds:  282.4691779716033
Frames per seconds:  276.87466500811354
Frames per seconds:  278.37295285758654
Frames per seconds:  277.65913632546784
mean_rewards:  -104.01173753970193 mean_steps:  87.5
Frames per seconds:  277.513692028919
Frames per seconds:  275.9347059631518
Frames per seconds:  275.3460039618072
Frames per seconds:  281.10269908423965
Frames per seconds:  278.09975264797333
mean_rewards:  -173.8081075881218 mean_steps:  79.2
Frames per seconds:  279.8647839453481
Frames per seconds:  283.3731155709269
Frames per seconds:  280.73487183750905
Frames per seconds:  281.82379481024253
Frames per seconds:  280.9903875040693
mean_rewards:  -385.1997048094497 mean_steps:  103.9
Frames per seconds:  275.4611794638439
Frames per seconds:  276.7035839088586
Frames per seconds:  275.89200650300717
Frames per seconds:  266.2309637832632
Frames per seconds:  292.085569822482
mean_rewards:  -227.4178400856844 mean_steps:  119.4
Frames per seconds:  304.6992355147931
Fra

In [ ]:
#agent.save("./nn/ENDDDQN2" + env_name)


In [ ]:
from ddpgagent import rescale_actions

import wrappers
#print(env.unwrapped.get_action_meanings())
def evaluate(env,t_max=10000):
    rewards = []
    #env._max_episode_steps = 200
    print('reset')
    #env = env.old_env
    s = env.reset()
    reward = 0
    for it in range(t_max):
        env.render()
        action = agent.get_action(s)
        action = rescale_actions(agent.actions_low, agent.actions_high, action)
        #print(action)
        s, r, done, _ = env.step(action)
        reward += r
        
            
        if done:
            break       
        
    return reward

import gym.wrappers


#env_monitor = gym.wrappers.Monitor(env,directory='video_dddqn05',force=True)

sessions = [print('reward:', evaluate(env)) for _ in range(10)]
env_monitor.close()

In [ ]:
#tf.reset_default_graph()
#agent = DQNAgent(env, sess, ExperienceBuffer(EXP_BUFFER_CAPACITY), env_name, config = dqn_config)

In [ ]:
#env.reset()
#agent.train()